In [1]:
import os
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def insert_depts(conn, values):
    sql = ''' INSERT INTO Departments(DepartmentName)
              VALUES(?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


def insert_student(conn, values):
    sql = ''' INSERT INTO Students(StudentName, DepartmentId, DateOfBirth)
              VALUES(?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, values)
    return cur.lastrowid


def get_dept_fk(dept):
    cur = conn.cursor()
    cur.execute("""SELECT DepartmentId FROM Departments WHERE DepartmentName == (?);""",  (dept, ))

    fk = cur.fetchone()
    #(1,)
    #(1,)[0] --> 1
    if fk:
        return fk[0]
    else:
        return None


def select_all_students(conn):
    cur = conn.cursor()
    cur.execute("""SELECT * FROM Students INNER JOIN Departments USING(DepartmentId);""")

    rows = cur.fetchall()

    for row in rows:
        print(row)

    return rows


In [13]:
db_file = 'sample_data_py3.db'
if os.path.exists(db_file):
    os.remove(db_file)

create_table_departments_sql = """ CREATE TABLE [Departments] (
    [DepartmentId] INTEGER  NOT NULL PRIMARY KEY,
    [DepartmentName] NVARCHAR(50)  NULL
); """

create_table_students_sql = """ CREATE TABLE [Students] (
    [StudentId] INTEGER  PRIMARY KEY NOT NULL,
    [StudentName] NVARCHAR(50) NOT NULL,
    [DepartmentId] INTEGER  NULL,
    [DateOfBirth] DATE NULL,
    FOREIGN KEY(DepartmentId) REFERENCES Departments(DepartmentId)
);  """

conn = create_connection(db_file)


depts = ('IT', 'Physics', 'Arts', 'Math')
students = (
    ('Michael', 'IT', '1998-10-12'),
    ('John', 'IT', '1998-10-12'),
    ('Jack', 'IT', '1998-10-12'),
    ('Sara', 'Physics', '1998-10-12'),
    ('Sally', 'Physics', '1998-10-12'),
    ('Jena', None, '1998-10-12'),
    ('Nancy', 'Physics', '1998-10-12'),
    ('Adam', 'Arts', '1998-10-12'),
    ('Stevens', 'Arts', '1998-10-12'),
    ('George', None, '1998-10-12')
)

with conn:

    create_table(conn, create_table_departments_sql)
    create_table(conn, create_table_students_sql)
    for values in depts:
        insert_depts(conn, (values, ))
    for values in students:
        values = list(values)
        # ['Michael', 'IT', '1998-10-12']
        print(values)
        values[1] = get_dept_fk(values[1])
        # ['Michael', 1, '1998-10-12']
        print(values)
        insert_student(conn, values)
   


['Michael', 'IT', '1998-10-12']
['Michael', 1, '1998-10-12']
['John', 'IT', '1998-10-12']
['John', 1, '1998-10-12']
['Jack', 'IT', '1998-10-12']
['Jack', 1, '1998-10-12']
['Sara', 'Physics', '1998-10-12']
['Sara', 2, '1998-10-12']
['Sally', 'Physics', '1998-10-12']
['Sally', 2, '1998-10-12']
['Jena', None, '1998-10-12']
['Jena', None, '1998-10-12']
['Nancy', 'Physics', '1998-10-12']
['Nancy', 2, '1998-10-12']
['Adam', 'Arts', '1998-10-12']
['Adam', 3, '1998-10-12']
['Stevens', 'Arts', '1998-10-12']
['Stevens', 3, '1998-10-12']
['George', None, '1998-10-12']
['George', None, '1998-10-12']


In [ ]:
 select_all_students(conn)

In [7]:
cur = conn.cursor()
cur.execute("""SELECT StudentName FROM Students INNER JOIN Departments USING(DepartmentId);""")

rows = cur.fetchall()
print(rows)


[('Michael',), ('John',), ('Jack',), ('Sara',), ('Sally',), ('Nancy',), ('Adam',), ('Stevens',)]


In [6]:
for row in rows:
    print(row)



(1, 'Michael', 1, '1998-10-12', 'IT')
(2, 'John', 1, '1998-10-12', 'IT')
(3, 'Jack', 1, '1998-10-12', 'IT')
(4, 'Sara', 2, '1998-10-12', 'Physics')
(5, 'Sally', 2, '1998-10-12', 'Physics')
(7, 'Nancy', 2, '1998-10-12', 'Physics')
(8, 'Adam', 3, '1998-10-12', 'Arts')
(9, 'Stevens', 3, '1998-10-12', 'Arts')


In [8]:
x = ('Michael', 'IT', '1998-10-12')
x

('Michael', 'IT', '1998-10-12')

In [9]:
x[1] = 2

TypeError: 'tuple' object does not support item assignment

In [11]:
y = list(x)
y[1] = 2
y

['Michael', 2, '1998-10-12']